In [ ]:
!pip install -q google-genai

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Exercise 5: Executing a Tool

In the previous exercise, the model asked us to call a tool. Now we'll complete the round-trip: execute the tool and send the result back.

By the end, you'll understand:
- How to append the model's function-call response to the contents list
- How to construct a `FunctionResponse` part with the result
- The full request/response cycle for tool use in Gemini

## Step 1: Setup (same as previous exercise)

In [ ]:
from google import genai
from google.genai import types

client = genai.Client()


def get_weather(location: str) -> str:
    """Get the current weather in a given location."""
    pass


config = types.GenerateContentConfig(
    tools=[get_weather],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
)

## Step 2: Get the tool-call response

Note: we use explicit `Content` objects so we can build the multi-turn conversation.

In [ ]:
contents = [types.Content(role="user", parts=[types.Part.from_text(text="What is the weather in Tokyo?")])]

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=contents,
    config=config,
)

model_content = response.candidates[0].content
print(f"Parts: {model_content.parts}")

## Step 3: Append the model's response

**Important:** You must append the full model content object. This preserves the `function_call` parts, which the API needs to match tool results back to their requests.

In [ ]:
contents.append(model_content)

## Step 4: Find the function call and "execute" the tool

In a real application, you'd call an actual weather API here. For now, we return a hardcoded result.

Note: `function_call.args` is already a **dict** — no JSON parsing needed!

In [ ]:
fc = model_content.parts[0].function_call

print(f"The model wants to call: {fc.name}")
print(f"With arguments: {fc.args}")

# "Execute" the tool (hardcoded for now)
tool_result = "65 degrees"

## Step 5: Send the tool result back

We send the result as a `Content` with a `FunctionResponse` part. The `name` must match the function call.

In [ ]:
contents.append(
    types.Content(
        role="user",
        parts=[types.Part.from_function_response(name=fc.name, response={"output": tool_result})],
    )
)

final_response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=contents,
    config=config,
)

print(final_response.text)

## Step 6: Review the full message flow

Let's look at the complete conversation to see the three-step pattern:
1. **User** asks a question
2. **Model** responds with a `function_call`
3. **User** sends back a `FunctionResponse`
4. **Model** gives the final natural language answer

In [ ]:
contents.append(final_response.candidates[0].content)

for msg in contents:
    print(f"--- {msg.role} ---")
    for part in msg.parts:
        if part.text:
            print(part.text)
        elif part.function_call:
            print(f"[function_call: {part.function_call.name}({part.function_call.args})]")
        elif part.function_response:
            print(f"[function_response: {part.function_response.name} → {part.function_response.response}]")
    print()

## Key takeaway

The tool-use cycle is:

**You → Model:** "What's the weather?" (+ tool definitions)  
**Model → You:** "Please call `get_weather` with `{"location": "Tokyo"}`"  
**You → Model:** "Here's the result: 65 degrees"  
**Model → You:** "The weather in Tokyo is 65 degrees."

In the next exercise, we'll automate this into a loop so it can handle multiple tool calls.